# Part 4a: Synthesis techniques - Additive synthesis

(Draft: May move around topics)

In part 4, you'll be introduced to some common audio synthesis techniques at a cursory level. Each of these topics is deep enough to spend an entire workshop on them. Our goal though is to get a broad enough understanding that we'll be able to start working with a specific approach -- physical modeling synthesis -- with this background. We start with the simplest of the techniques -- known as additive synthesis.

## The idea of an "instrument" that is used in a "performance"

When thinking about humans making music, we can easily conceive of a person "playing" and "instrument". The instrument has some sonic characteristics that we attribute to the physicality of the instrument and something we don't expect to change qualitatively as the musician "plays" it. The instrument also offers modes of control that permit the musician to make meaningful sounds with it -- thus "performing" with it.

When dealing with computer synthesized sounds for music making, there is virtually no limit on the kinds of sounds that can be produced, ignoring the difficulty of producing them. This is because we can compute every sample of the output sound using whatever method we want to. We can produce pure white noise, for example, by setting each sample to a random value in the range $[-1.0,1.0]$ ... something that we cannot do with physical instruments.

The downside of this possibility expansion is ... the very large space of possibilities. To bring this under a bit of control, we can choose to separate concerns by building a function that will serve as an "instrument" which can then be "performed" by setting the value of some control parameters to vary in a certain way over time. The idea here is to make the instrument have a consistent identifiable characteristic to it, though that is not a necessity if you want to experiment.

## Our "sine wave" instrument

We've already made such an instrument -- the "sine wave" function -- which can be controlled by varying its amplitude and frequency over time.

In [1]:
function sine_wave(amplitude, frequency, sampling_rate=48000)
    p = 0.0f0
    function wave(dt)
        pval = p
        p = p + frequency(dt) * dt
        a = amplitude(dt)
        a * sin(2.0f0 * pi * p)
    end
end

sine_wave (generic function with 2 methods)

The `amplitude` and `frequency` arguments are in our control. These are too complex for us to control in real time sample to sample though and it would be nice to have to specify far fewer things. 

Now, in the natural world, even including humans, a pure sine tone that lasts for ever does not occur at all. In as far as pure tones do occur (ex: ringing metals), they decay over time and become silent after a while. We can model such a decay by setting the amplitude to a time varying value that is shaped like an exponential function $a e^-kt$.

In [21]:
function konst(val)
    function wave(dt)
        val
    end
end

function decay(peak, dur)
    t = 0.0
    k = 3.0   # The exponential will decay from the given
              # peak value down to 2^-3 over dur seconds.
    function wave(dt)
        tval = t
        t = t + dt
        peak * (2 ^ (-k * t / dur))
    end
end

function decaying_sine(peak, freq, dur, sampling_rate=48000)
    envelope = decay(peak, dur)
    sine_wave(envelope, konst(freq), sampling_rate)
end


decaying_sine (generic function with 2 methods)

There are some slight "problems" with the decaying sine wave. For one thing, it does not come down to 0 after the given duration, so the duration is more like a suggestion to the function. If we want it to go down to zero strictly speaking, what we can do is $$\frac{2^{-k\min(1,t/d)} - 2^{-k}}{1 - 2^{-k}}$$. Though such an envelope isn't usually a product of any physical process, it approximates an exponential decay while also meeting the convenient criterion of going down to 0 after a finite duration.

We can now use such a decaying sine wave to make a "tune player".

In [25]:
"""
    seq(dursig)

dursig is an array of pairs - whose first values are
durations in seconds and whose second values are signals
that have to be played for those specific durations.
The result is a composite tone "sequence".
"""
function seq(dursig)
    t = 0.0f0
    i = 1
    function wave(dt)
        if i < length(dursig)
            t = t + dt
            val = dursig[i][2](dt)
            if t >= dursig[i][1]
                t = t - dursig[i][1]
                i = i + 1
            end
            val
        else
            # If the sequence has ended, just keep playing the
            # last signal.
            dursig[end][2](dt)
        end
    end
end
    
pitch2freq(p) = 440 * 2 ^ ((p - 69)/12)

function tune()
    # The melodic scale we're going to use.
    scale = 60 .+ [0, 2, 4, 5, 7, 9, 11, 12]
    
    # The pattern of notes from the scale that we're going to play.
    pat = [1, 2, 3, 1, 3, 1, 3, 2, 3, 4, 4, 3, 2, 4]
    dur = 0.25 * [3, 1, 3, 1, 2, 2, 4, 3, 1, 1, 1, 1, 1, 8]
    
    # Compute the amplitude shape of each note.
    amp = seq([(dur[i], decay(0.3, dur[i])) for i in 1:length(pat)])
    
    # Compute the frequency of each note.
    freq = seq([(dur[i], konst(pitch2freq(scale[pat[i]]))) for i in 1:length(pat)])
    
    # Pass the computed amp and freq to the sine wave oscillator to make
    # the result. Note that we're using a single oscillator and controlling
    # it to play a tune.
    (sine_wave(amp, freq), sum(dur))
end

tune (generic function with 1 method)

We can now test out the tune player above by writing its output to a file.

In [24]:
open("/tmp/tune.float32", "w") do out
    w = tune()
    dt = 1/48000
    write(out, [convert(Float32, w[1](dt)) for t in 0:dt:w[2]])
end

1536004

## Task

Listen to the output and note down your observations about how it sounds, what parts you think are ok and what parts aren't.

We used a single oscillator (`sine_wave`) to play a tune. What do you think are the limitations of this approach? How would you address these limitations?

Can you try the same approach using triangle and other wave functions you've written earlier?

## Additive synthesis

From Fourier analysis, we know that any given periodic signal can be decomposed into a set of sine and cosine signals. This forms the basis of what is known as "additive synthesis" - where a compound signal with potentially varying tonal quality (or "timbre") is constructed by adding together a number of sine and cosine waves and manipulating their amplitudes.

While technically you have full freedom to decide the shape of the amplitudes and frequencies of these signals, the idea is to shape them in a manner that yields a "coherent" sound you can use for musical purposes. For example, periodic and quasi-periodic [^quasi] signals can be decomposed into a sum of sine/cosine functions with frequencies being integer multiples of the signal's frequency. These other "integer multiple frequencies" of the base frequency go by the name "harmonics".

So a subset of additive synthesis may also be stated as constructing signals as a sum of a set of shaped harmonics. 

An example of the kind of shaping is that in real instruments, higher frequencies tend to decay at a faster rate than lower frequencies. So we could construct the decay rate of each harmonic as a function of its frequency, thereby creating an illusion of cohesiveness in the result sound.

[^quasi]: "quasi" is often used to mean "pseudo" or "approximately" and usually refers to some conditions required for the original concept being relaxed.